In [ ]:
# third party
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy
from syft.abstract_node import NodeType
from syft.service.code.user_code import UserCodeStatus
from syft.service.network.node_peer import NodePeer
from syft.service.network.routes import HTTPNodeRoute
from syft.service.project.project import ProjectCode
from syft.service.project.project import check_route_reachability
from syft.service.response import SyftSuccess
from syft.types.uid import UID

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_HOST = None
CANADA_ENCLAVE_PORT = 9083

CPU_ENCLAVE = "13.90.101.161"
GPU_ENCLAVE = "172.176.204.136"
#! Uncomment below line to run the code on a pre-provisioned remote Enclave
CANADA_ENCLAVE_HOST = CPU_ENCLAVE

# Launch nodes

We will begin by launching two domain nodes and an enclave node.

### For Kubernetes
To run the nodes in kubernetes, run the below commands and wait till the cluster becomes ready.
```bash
CLUSTER_NAME=canada-domain CLUSTER_HTTP_PORT=9081 tox -e dev.k8s.launch.domain
CLUSTER_NAME=italy-domain CLUSTER_HTTP_PORT=9082 tox -e dev.k8s.launch.domain
CLUSTER_NAME=canada-enclave CLUSTER_HTTP_PORT=9083 tox -e dev.k8s.launch.enclave
```

In [ ]:
canada_node = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_node = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
enclave_kwargs = {
    "name": "canada-enclave",
    "node_type": NodeType.ENCLAVE,
    "port": CANADA_ENCLAVE_PORT,
    "create_producer": True,
    "n_consumers": 3,
    "dev_mode": True,
    "reset": True,
}
if CANADA_ENCLAVE_HOST:
    enclave_kwargs.update({"deploy_to": "remote", "host": CANADA_ENCLAVE_HOST})

canada_enclave = sy.orchestra.launch(**enclave_kwargs)

In [ ]:
do_canada_client = canada_node.login(email="info@openmined.org", password="changethis")
do_italy_client = italy_node.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.node_type == NodeType.DOMAIN
assert do_italy_client.metadata.node_type == NodeType.DOMAIN

# Upload datasets to both domains

In [ ]:
# Using public datasets from Freely Extensible Biomedical Record Linkage (Febrl) project
canada_census_data, italy_census_data = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data, italy_census_data],
    [do_canada_client, do_italy_client],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

In [ ]:
assert len(do_canada_client.datasets.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

# Create account for data scientist on both the domains

In [ ]:
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

# Register the enclave with Canada domain

In [ ]:
route = HTTPNodeRoute(host_or_ip=canada_enclave.url, port=canada_enclave.port)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

## Login to DS Accounts

In [ ]:
ds_canada_client = canada_node.login(email="sheldon@caltech.edu", password="changethis")
ds_italy_client = italy_node.login(email="sheldon@caltech.edu", password="changethis")

## Create Association Requests

In [ ]:
canada_node_peer = NodePeer.from_client(ds_canada_client)
canada_node_peer

In [ ]:
italy_node_peer = NodePeer.from_client(ds_italy_client)
italy_node_peer

In [ ]:
canada_conn_req = ds_canada_client.api.services.network.add_peer(italy_node_peer)
canada_conn_req

In [ ]:
italy_conn_req = ds_italy_client.api.services.network.add_peer(canada_node_peer)
italy_conn_req

In [ ]:
do_canada_client.requests[-1].approve()

In [ ]:
do_italy_client.requests[-1].approve()

In [ ]:
check_route_reachability([ds_canada_client, ds_italy_client])

# Find datasets across multiple domains

In [ ]:
canada_census_data = ds_canada_client.datasets[-1].assets[0]
italy_census_data = ds_italy_client.datasets[-1].assets[0]

In [ ]:
# find available enclaves
all_enclaves = ds_canada_client.enclaves.get_all() + ds_italy_client.enclaves.get_all()
all_enclaves

In [ ]:
enclave = all_enclaves[0]
enclave

# Create and submit a distributed project

In [ ]:
# Code to perform the multi-party computation


@sy.syft_function(
    input_policy=sy.ExactMatch(
        canada_census_data=canada_census_data,
        italy_census_data=italy_census_data,
    ),
    output_policy=sy.SingleExecutionExactOutput(),
    runtime_policy=sy.RunOnEnclave(
        provider=enclave,
        # image=sy.DockerWorkerConfig(dockerfile=dockerfile_str),
        # workers_num=4,
        # worker_pool_name=worker_pool_name,
        # timeout=300,
        # result_persistence={"storage_path": "/data/enclave", "retention_policy": "30d"}
    ),
)
def compute_census_matches(canada_census_data, italy_census_data):
    # third party
    import recordlinkage

    # Index step
    indexer = recordlinkage.Index()
    indexer.block("given_name")

    candidate_links = indexer.index(canada_census_data, italy_census_data)

    # Comparison step
    compare_cl = recordlinkage.Compare()

    compare_cl.exact("given_name", "given_name", label="given_name")
    compare_cl.string(
        "surname", "surname", method="jarowinkler", threshold=0.85, label="surname"
    )
    compare_cl.exact("date_of_birth", "date_of_birth", label="date_of_birth")
    compare_cl.exact("suburb", "suburb", label="suburb")
    compare_cl.exact("state", "state", label="state")
    compare_cl.string("address_1", "address_1", threshold=0.85, label="address_1")

    features = compare_cl.compute(
        candidate_links, canada_census_data, italy_census_data
    )

    # Classification step
    matches = features[features.sum(axis=1) > 3]

    return len(matches)

In [ ]:
# Check result of execution on mock data
mock_result = compute_census_matches(
    canada_census_data=canada_census_data.mock,
    italy_census_data=italy_census_data.mock,
)
mock_result

In [ ]:
new_project = sy.Project(
    name="Census Matching",
    description="Match census data between Canada and Italy",
    members=[ds_canada_client, ds_italy_client],
)
new_project

In [ ]:
project = new_project.send()
project

In [ ]:
project.create_code_request(
    compute_census_matches, clients=[ds_canada_client, ds_italy_client]
)

In [ ]:
assert len(do_canada_client.code.get_all()) == 1
assert len(do_italy_client.code.get_all()) == 1

In [ ]:
canada_project = do_canada_client.projects[0]
canada_code_event = canada_project.events[0]
assert isinstance(canada_code_event, ProjectCode)
canada_code_event.status(canada_project, verbose=True)

In [ ]:
canada_code_request = [
    r for r in do_canada_client.requests if isinstance(r.code_id, UID)
][-1]
assert canada_code_request.code_id == compute_census_matches.id
canada_code_request.approve()
canada_project.sync()
canada_code_event.status(canada_project, verbose=True)

In [ ]:
italy_project = do_italy_client.projects[0]
italy_code_event = italy_project.events[0]
assert isinstance(italy_code_event, ProjectCode)
italy_code_event.status(italy_project, verbose=True)

In [ ]:
italy_code_request = [
    r for r in do_italy_client.requests if isinstance(r.code_id, UID)
][-1]
assert italy_code_request.code.id == compute_census_matches.id
italy_code_request.approve()
italy_project.sync()
italy_code_event.status(italy_project, verbose=True)

In [ ]:
canada_project = do_canada_client.projects[0]
italy_project = do_italy_client.projects[0]
assert canada_project.id == italy_project.id

In [ ]:
assert canada_project.events[0].status(canada_project) == UserCodeStatus.APPROVED
assert italy_project.events[0].status(italy_project) == UserCodeStatus.APPROVED

In [ ]:
code = project.code[0]

In [ ]:
code.setup_enclave()

In [ ]:
code.view_attestation_report(attestation_type="CPU")

In [ ]:
code.request_asset_transfer()

In [ ]:
code.request_execution()

In [ ]:
result = code.get_result()
print(result)

In [ ]:
# Or you can call all of the above in one line using the following
result = code.orchestrate_enclave_execution()
print(result)

# Cleanup local domain servers

In [ ]:
if canada_node.deployment_type.value == "python":
    canada_node.land()

if italy_node.deployment_type.value == "python":
    italy_node.land()

if canada_enclave.deployment_type.value == "python":
    canada_enclave.land()